# 심층 신경망 훈련하기

✔ 그레이디언트 소실, 그레이디언트 폭주 문제 해결하기

- weight initialization(글로럿과 He 초기화) -> 이 부분은 학교에서 배웠기에 넘어감

- 다른 activation function을 사용하기

- batch normalization 사용하기

- 그레이디언트 폭주 문제를 해결하기 위해서 learning rate를 낮추기


#  ✔ Initialization 예제 코드

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root = 'MNIST_data/', train =True, transform = transforms.ToTensor(), download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/', train = False, transform = transforms.ToTensor(), download = True)

data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle=True, drop_last = True)

linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

#weight initialization section
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

total_batch = len(data_loader)
for epoch in range(training_epochs) :
  avg_cost = 0

  for X, Y in data_loader :
    X = X.view(-1, 784).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch: ', '%04d' % (epoch + 1), 'cost=', '{:.9f}'.format(avg_cost))
  print('Learning finished')

  with torch.no_grad() :
    X_test = mnist_test.test_data.view(-1, 784).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())





# ✔ batch normalization 예제 코드

In [ ]:
bn1 = nn.BatchNormal1d(32)
bn2 = nn.BatchNormal1d(32)

# ✔ 그레이디언트 클리핑 예제 코드

✔ 그레이디언트 폭주 문제를 완화하는 방법으로 역전파가 될 때 일정 임곗값을 넘어서지 못하게 그레이디언트를 잘라내는 방법

✔ 만약 그레이디언트 벡터의 모든 원소를 -1.0과 1.0 사이로 클리핑한다면 모든 편미분 값을 -1.0에서 1.0 사이로 잘라낸다.

In [ ]:
learning_rate = 1.
max_grad_norm = 5.

optimizer = optim.SGD(model.parameters(), lr=learning_rate)
torch_utils.clip_grad_norm_(model.parameters(), max_grad_nom)

optimizer.step()


# ✔ 옵티마이저(충분히 배운 내용)

- 확률적 경사 하강법(sgd)
- 모멘텀 최적화:
처음에는 천천히, 경사가 급해지면 빠르게
- 네스테로프 가속 경사:
모멘텀 최적화의 한 변종으로 기본 모멘텀 최적화보다 빠름, 방향 역시 고려하기 때문에 더 빠르게 골짜기로 잡아당긴다.
- AdaGrad:
가장 가파른 차원을 따라 그레이디언트 벡터의 스케일을 감소시켜 문제를 해결함
- RMSProp:
AdaGrad에 감쇠율 적용 
- Adam:
모멘텀 최적화 + RMSProp 아이디어 합치기


# ✔ scheduler

- 학습 스케줄
큰 학습률로 시작하고 학습 속도가 느려질 때 학습률을 낮추며 좋은 솔루션을 더 빨리 발견하는 방법
- decay는 학습률을 나누기 위해 수행할 스텝 수의 역수

In [ ]:
def exponential_decay_fn(epoch) :
  return 0.01 * 0.1 **(epoch/20)

def exponential_decay(lr0, s) :
  def exponential_decay_fn(epoch) :
    return lr0 * 0.1 ** (epoch / 2) 
  return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0 = 0.01, s=20)


# ✔ regularization

- l1과 l2 규제
-dropout:
특정 뉴런의 일부를 제거하는 것
- 맥스-노름 규제: 규제 손실 항을 추가하지 않고 일반적으로 훈련 스텝이 끝나고 |w|를 계산하고 필요하면 w의 스케일을 조정함.


In [ ]:
dropout = torch.nn.Dropout(p=drop_prob)

a = torch.arange(9, dtype = torch.float) - 4
max_norm = torch.norm(a)